Creation d'un classe ? ce fait avec class "type"

In [9]:
class MyClass:
    def method(self):
        return 1
        
m = MyClass()
print(m.method())
print(MyClass)
print(type(MyClass))
print(MyClass.__dir__)

1
<class '__main__.MyClass'>
<class 'type'>
<method '__dir__' of 'object' objects>


On utilise le constructeur de type pour créer MyClass2 sans la syntaxe "class"

In [49]:
def method(self):
        return 1
    
MyClass2 = type("MyClass2", (object,), {"method":method})
m2 = MyClass2()
print(m2.method())
print(MyClass2)
print(type(MyClass2))
print(MyClass2.__dir__)

1
<class '__main__.MyClass2'>
<class 'type'>
<method '__dir__' of 'object' objects>


In [11]:
class LOLEALFSKTGEOTEOTLKGKEGZKDGKGK:
    pass
print(LOLEALFSKTGEOTEOTLKGKEGZKDGKGK)

<class '__main__.LOLEALFSKTGEOTEOTLKGKEGZKDGKGK'>


Override type pour faire une classe qui crée des classes avec des proprietes spécifiques

In [51]:
class RevealingMeta(type):
    @classmethod
    def __prepare__(mcs, name, bases, **kwargs):
        print(f"{mcs}, __prepare__ is called")
        return super().__prepare__(name, bases, **kwargs)
    def __new__(mcs, name, bases, namespace, **kwargs):
        print(f"{mcs}, __new__ is called")
        return super().__new__(mcs, name, bases, namespace)
    def __init__(cls, name, bases, namespace, **kwargs):
        print(f"{cls}, __init__ is called")
        return super().__init__(name, bases, namespace)

    
    def __call__(cls, *args, **kwargs):
        print(f"{cls}, __call__ is called")
        return super().__call__(*args, **kwargs)


In [41]:
class MyClassMeta(metaclass = RevealingMeta):
    def __init__(self):
        print(f"{self} __init__ is called")
        super().__init__()
        
    def __new__(cls):
        print(f"{cls} __new__ is called")
        return super().__new__(cls)

<class '__main__.RevealingMeta'>, __prepare__ is called
<class '__main__.RevealingMeta'>, __new__ is called
<class '__main__.MyClassMeta'>, __init__ is called


In [42]:
m = MyClassMeta()

<class '__main__.MyClassMeta'>, __call__ is called
<class '__main__.MyClassMeta'> __new__ is called
<__main__.MyClassMeta object at 0x0000018179EC9520> __init__ is called


La métaclasse qui va permettre de compter l'ensemble des instances de la accessible via compteur

In [56]:
class CoutingMeta(type):
    compteur = 0
    @classmethod
    def __prepare__(mcs, name, bases, **kwargs):
        print(f"{mcs}, __prepare__ is called")
        return super().__prepare__(name, bases, **kwargs)
    def __new__(mcs, name, bases, namespace, **kwargs):
        print(f"{mcs}, __new__ is called")
        return super().__new__(mcs, name, bases, namespace)
    def __init__(cls, name, bases, namespace, **kwargs):
        print(f"{cls}, __init__ is called")
        return super().__init__(name, bases, namespace)

    
    def __call__(cls, *args, **kwargs):
        cls.compteur +=  1
        print(f"{cls}, __call__ is called, {cls.compteur} instance created")
        return super().__call__(*args, **kwargs)


In [57]:
class CoutingClass(metaclass = CoutingMeta):
    pass

<class '__main__.CoutingMeta'>, __prepare__ is called
<class '__main__.CoutingMeta'>, __new__ is called
<class '__main__.CoutingClass'>, __init__ is called


In [58]:
cc = CoutingClass()

<class '__main__.CoutingClass'>, __call__ is called, 1 instance created


In [59]:
cc1 = CoutingClass()

<class '__main__.CoutingClass'>, __call__ is called, 2 instance created


MetaClass pour garder une sauvegarde à PluginMeta.plugins pour les classes qui implémentent PluginBase

In [60]:
class PluginMeta(type):
    plugins = {}
    def __new__(mcs, name, bases, namespace):
            cls = super().__new__(mcs, name, bases, namespace)
            if name != 'PluginBase':
                    # Enregistrement automatique du plugin
                    PluginMeta.plugins[name] = cls
            return cls

class PluginBase(metaclass=PluginMeta):
    pass
    
class HealthPlugin(PluginBase):
    pass
class InventoryPlugin(PluginBase):
    pass
# Les plugins sont automatiquement enregistrés et peuvent être accédés
print(PluginMeta.plugins)  # Affiche les plugins enregistrés

{'HealthPlugin': <class '__main__.HealthPlugin'>, 'InventoryPlugin': <class '__main__.InventoryPlugin'>}


MetaClass qui permet de rendre un membre abstrait

In [63]:
from abc import ABC, abstractmethod

class MyBaseClass(ABC):
    @abstractmethod
    def do_something(self):
        pass
        
class ConcreteClass(MyBaseClass):
    def do_something(self):
        print("Do smothing")

class TESTClass(MyBaseClass):
    pass

In [64]:
m = TESTClass()

TypeError: Can't instantiate abstract class TESTClass without an implementation for abstract method 'do_something'

In [78]:
class AbstractMetaClass(type):
    def __new__(mcs, name, bases, namespace):
            print(namespace)
            if "abstract_methods" in namespace:
                for method in namespace["abstract_methods"]:
                    print(method, " is tested")
                    function = namespace.get(method)
                    print(function)
                    if not callable(function):
                        raise VauleError(f"{method} not implemented")
            return super().__new__(mcs, name, bases, namespace)
        
class BaseClass(metaclass = AbstractMetaClass):
    abstract_methods = ["do_something"]
    def do_something(self):
        pass
        
class ConcreteClass(MyBaseClass):
    def do_something(self):
        print("Do smothing")
        
class TESTClass(MyBaseClass):
    pass

{'__module__': '__main__', '__qualname__': 'BaseClass', 'abstract_methods': ['do_something'], 'do_something': <function BaseClass.do_something at 0x000001817A42E160>}
do_something  is tested
<function BaseClass.do_something at 0x000001817A42E160>
